In [25]:
META_DIR="/kaggle/repo/output/infer/vlsp2023emo"

!ls $META_DIR/*/ -d

/kaggle/repo/output/infer/vlsp2023emo/11_26_adapt_frozen_emo_enc_only/
/kaggle/repo/output/infer/vlsp2023emo/11_28_adapt_freeze_emo_end_mfa/
/kaggle/repo/output/infer/vlsp2023emo/11_29_emofs2mas_fixed/
/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc/
/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/
/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-emo_enc+mha+text_enc/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP14-emofs2+shuffle_ref_mel+with_aug-120k/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP14-emofs2+shuffle_ref_mel+without_aug-120k/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP14-emofs2+shuffle_ref_mel+without_aug-60k/
/kaggle/repo/output/infer/vlsp2023emo/12-05-EXP15-multi-speaker-baseline-50k/
/kaggle/repo/output/infer/vlsp2023emo/12-07-EXP17_multi_speaker_move_emo_enc_819177ad-50k/
/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/
/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_81

In [43]:
meta_dic = {
    "finetune+giangoi": f"{META_DIR}/12-01-freeze-all-enc-GiangOi2/",
    "multi_spk+giangoi": f"{META_DIR}/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2/",
    "multi_spk+move_emo+giangoi": f"{META_DIR}/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2/",
    # ""
}
meta_dic

{'finetune+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/',
 'multi_spk+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2/',
 'multi_spk+move_emo+giangoi': '/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2/'}

In [41]:
import os
import pandas as pd

def get_files(directory, meta_name="finetune+giangoi"):
    spk_name = meta_name.split("+")[-1].upper()
    assert spk_name in ["GIANGOI", "LMH", "TTH"]
    
    files = os.listdir(directory)
    fpaths = [os.path.join(directory, f) for f in files]
    names = [f.split(".")[0].split('-') for f in files]
    fids = [n[0] for n in names]
    femo = [n[2] for n in names]
    df = pd.DataFrame({"fid": fids, "femo": femo})
    df['fspk'] = spk_name
    df['fpath'] = fpaths
    
    return df

df = get_files(meta_dic["finetune+giangoi"])
df.head()

,fid,femo,fspk,fpath
0,emo_val_001120,angry,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...
1,emo_val_001120,happy,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...
2,emo_val_001120,neutral,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...
3,emo_val_001120,sad,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...
4,emo_val_001120,surprise,GIANGOI,/kaggle/repo/output/infer/vlsp2023emo/12-01-fr...


In [112]:
from types import SimpleNamespace


audio_dirs = []
for meta_name, directory in meta_dic.items():
    audio_dirs.append((meta_name, directory))

args = SimpleNamespace(
    title="mos_naturalness_finetune_vs_multi_spk",
    audio_dirs=audio_dirs,
    clean=True,
    max_file_per_exp=9,
)

print("Args: ")
print(f"\tTitle: {args.title}")
print("\tExperiments dirs: ")
for idx, d in enumerate(args.audio_dirs):
    print(f"\t\t{d[0]}: {d[1]}")
print(f"\tClean: {args.clean}")
print(f"\tMax file per exp: {args.max_file_per_exp}")

Args: 
	Title: mos_naturalness_finetune_vs_multi_spk
	Experiments dirs: 
		finetune+giangoi: /kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/
		multi_spk+giangoi: /kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2/
		multi_spk+move_emo+giangoi: /kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2/
	Clean: True
	Max file per exp: 9


3

In [113]:
import os
import shutil
import argparse
import random
import json
from itertools import chain


out_audio_dir = "./wavs"
out_cfg_dir = "./configs"
max_file_per_exp=args.max_file_per_exp
symmetry = True

exp_dic = {e[0]: e[1] for e in args.audio_dirs}
assert len(exp_dic.keys()) == len(args.audio_dirs), "Experiment name must be unique!"

if args.clean and os.path.exists(out_audio_dir):
    print("Clean wavs folder!")
    shutil.rmtree(out_audio_dir)

os.makedirs(out_audio_dir, exist_ok=True)

print("Audio dirs: ")
n_files = None
for idx, exp_name in enumerate(exp_dic):
    d = exp_dic[exp_name]
    files = os.listdir(d)
    files = [f for f in files if f.endswith(".wav")]
    assert len(files) != 0, d + " does not contain .wav files!" 
    assert not n_files or len(files) == n_files
    if symmetry:
        n_files = len(files)
    print(f"\tExperiment: {exp_name}: {len(files)} files")


Clean wavs folder!
Audio dirs: 
	Experiment: finetune+giangoi: 50 files
	Experiment: multi_spk+giangoi: 50 files
	Experiment: multi_spk+move_emo+giangoi: 50 files


In [120]:
selected = range(1, n_files)
n_emo = 5
n_text = int(len(selected) / n_emo)
text_range = list(range(1, n_text+1))

text_range

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [193]:
if symmetry and max_file_per_exp < n_files:
    # selected = random.sample(text_range, max_file_per_exp)
    # selected = sorted(selected)
    selected = [i*n_emo+random.randint(0, n_emo-1) for i in range(0, n_text)]
qids = list(range(1, len(args.audio_dirs)*len(selected)+1))
random.shuffle(qids)
selected

[3, 6, 10, 19, 21, 29, 34, 37, 40]

In [197]:
print("Copying...")
selected_dic = {}
for d_idx, exp_name in enumerate(exp_dic):
    d = exp_dic[exp_name]
    files = os.listdir(d)
    for flid, fid in enumerate(selected):
        if flid == 0 and not os.path.exists(os.path.join(out_audio_dir, exp_name)):
            os.makedirs(os.path.join(out_audio_dir, exp_name), exist_ok=True)
        f = files[fid]
        if exp_name not in selected_dic:
            selected_dic[exp_name] = []
        new_fname = f"{exp_name}/{f}"
        selected_dic[exp_name].append({
            "_new_fpath": new_fname,
            "fname": f,
            "fid": fid,
            "qid": f"q{qids[d_idx * len(selected) + flid]}",
        })
        shutil.copyfile(os.path.join(d, f), os.path.join(out_audio_dir, new_fname))


Copying...


In [198]:

print("Create config")
try:
    last_form_id = len(os.listdir(out_cfg_dir))
except Exception:
    last_form_id = 1
form_id = last_form_id+1
out_cfg = {
    "page_title": args.title, 
    "form_id": form_id, 
    "questions": [],
    "_meta": {
        "max_file_per_exp": max_file_per_exp,
        "symmetry": symmetry,
        "experiments": exp_dic,
        "selected_dic": selected_dic
    }
}
for idx, dic in enumerate(sorted(list(chain.from_iterable(selected_dic.values())), key=lambda d: int(d['qid'].replace("q", "")))):
    audio = dic['_new_fpath']
    del dic['_new_fpath']
    dic = {k: v for k, v in dic.items() if k != "_new_fpath"}
    name = dic['qid']
    audio_path = os.path.join(out_audio_dir, audio)
    title = f"Câu hỏi {idx+1}"
    out_cfg['questions'].append({
        "name": name,
        "audio_path": audio_path,
        "title": title
    })

print("Id: ", out_cfg['form_id'])
print("N question: ", len(out_cfg["questions"]))

os.makedirs(out_cfg_dir, exist_ok=True)
with open(os.path.join(out_cfg_dir, f"{form_id}.json"), "w") as f:
    f.write(json.dumps(out_cfg, indent=4))

print("Done!")

Create config
Id:  9
N question:  27
Done!


In [62]:
# # mos test natualness

# !python create_mos_config.py \
#     -a "finetune+giangoi" "/kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/" \
#     -a "multi_spk+giangoi" "/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2" \
#     -a "multi_spk+move_emo+giangoi" "/kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2" \
#     -c -t "mos_naturalness_finetune_vs_multi_spk" -m 10

Args: 
	Title: mos_naturalness_finetune_vs_multi_spk
	Experiments dirs: 
		finetune+giangoi: /kaggle/repo/output/infer/vlsp2023emo/12-01-freeze-all-enc-GiangOi2/
		multi_spk+giangoi: /kaggle/repo/output/infer/vlsp2023emo/12-13-EXP15-multi-speaker-baseline-150k/GiangOi2
		multi_spk+move_emo+giangoi: /kaggle/repo/output/infer/vlsp2023emo/12-13-EXP17_multi_speaker_move_emo_enc_819177ad-150k/GiangOi2
	Clean: True
	Max file per exp: 10

Creating...
Clean wavs folder!
Audio dirs: 
	Experiment: finetune+giangoi: 50 files
	Experiment: multi_spk+giangoi: 50 files
	Experiment: multi_spk+move_emo+giangoi: 50 files
Copying...
Traceback (most recent call last):
  File "/home/kryo/Desktop/tts-eval/create_mos_config.py", line 78, in <module>
    "qid": f"q{qids[d_idx * len(selected) + flid]}",
IndexError: list index out of range
